In [1]:
##学习神经网络之前需要安装三个包：
#pip install tensorflow
# conda install pip
# pip install keras
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA


#画出决策树-graphviz
import os
os.environ["PATH"]+=os.pathsep+"D:/CHEN/biancheng/JupyterTest/graphviz-2.38/release/bin/"
import pydotplus

def hr_preprocessing(sl=False,le=False,npr=False,amh=False,tsc=False,wa=False,pl5=False,dp=False,slr=False,lower_d=False,ld_n=1):
    df=pd.read_csv('./data/HR.csv')
   
    #1.清洗数据(去除异常值、抽样)
    df=df.dropna(subset=['satisfaction_level','last_evaluation'])
    df=df[df['satisfaction_level']<=1][df['salary']!='nme']
     #2。得到标注
    label = df['left']
    df=df.drop('left',axis=1)#axis=1指定以列删除
    #3.特征选择
    #4.特征处理  
    scaler_lst=[sl,le,npr,amh,tsc,wa,pl5]
    column_lst=['satisfaction_level','last_evaluation','number_project','average_monthly_hours','time_spend_company','Work_accident','promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=\
                MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]#二维格式
        else:
            df[column_lst[i]]=\
                StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
      
    scaler_lst=[dp,slr]
    column_lst=['department','salary']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i]=='salary':
                df[column_lst[i]]=[map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]]=LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df=pd.get_dummies(df,columns=[column_lst[i]])
    if lower_d:
        return PCA(n_components=ld_n).fit_transform(df,values).label#PVA降维，可不使用标注
    return df,label
#使salary返回对应的等级而不是012
d=dict([('low',0),('medium',1),('high',2)])
def map_salary(s):
    return d.get(s,0)
#建模
def hr_modeling(features,label):
    from sklearn.model_selection import train_test_split  #切分训练集测试集
    f_v=features.values
    f_names=features.columns.values #得到特征名称
    l_v=label.values
    X_tt,X_validation,Y_tt,Y_validation=train_test_split(f_v,l_v,test_size=0.2)   #获得验证集占20%
    X_train,X_test,Y_train,Y_test=train_test_split(X_tt,Y_tt,test_size=0.25)  #训练集、测试集
    #print(len(X_train),len(X_validation),len(X_test))#得到6：2：2
    #KNN
    from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
    from sklearn.metrics import accuracy_score,recall_score,f1_score#F一
    from sklearn.naive_bayes import GaussianNB,BernoulliNB#离散型条件下表现更好，对特征有严格要求
    #决策树包
    from sklearn.tree import DecisionTreeClassifier,export_graphviz
    from sklearn.externals.six import StringIO
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.linear_model import LogisticRegression  #引入逻辑回归
    #深度神经网络
    from keras.models import Sequential #序列化的容器
    from keras.layers import Dense,Activation  #稠密层
    from keras.optimizers import SGD  #随机梯度下降算法
    mdl=Sequential()
    #建一个稠密层表示输入，传入参数表示输出就可以了，只有在输入层表示输入，输出代表隐含层神经元个数
    #inpout_dim:输入维度与特征个数密切相关
    mdl.add(Dense(50,input_dim=len(f_v[0])))
    mdl.add(Activation('sigmoid'))
    mdl.add(Dense(2))#输出层  标注只有两个维度
    mdl.add(Activation('softmax'))#保证归一化
    sgd=SGD(lr=0.01)#构造优化器，lr:学习率--同α#lr=0.03/0.05有了召回，准确率增加些
    #编译 ，loss最优化函数：对每个输出减去对应输出值平方再开方，得到的值让它最小
    #mdl.compile(loss='mean_squared_error',optimizer=sgd) #优化器adam(亚当优化器)
    mdl.compile(loss='mean_squared_error',optimizer='adam')
    #nb_epoch迭代代数，batch_size下降梯度#np_epoch=10000,batch_size=8999
    #mdl.fit(X_train,np.array([[0,1] if i==1 else [1,0] for i in Y_train]),nb_epoch=1000,batch_size=2048)
    mdl.fit(X_train,np.array([[0,1] if i==1 else [1,0] for i in Y_train]),nb_epoch=10000,batch_size=8999)
    #预测
    xy_lst=[(X_train,Y_train),(X_validation,Y_validation),(X_test,Y_test)]#依次用0，1，2表示
    ##分类评估ROC、AUC
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve,auc,roc_auc_score
    f=plt.figure()
    for i in range(len(xy_lst)):
        X_part=xy_lst[i][0]
        Y_part=xy_lst[i][1]
        Y_pred=mdl.predict(X_part)#输出分类标注
        print(Y_pred)
        #onehot类型，只需要取出值为1数据，（-1，1）指的是一行，只需要在第一列调配
        Y_pred=np.array(Y_pred[:,-1]).reshape((-1,1))[0]
        
#         print(i)
#         print(NN,'-ACC',accuracy_score(Y_part,Y_pred))
#         print(NN,'-REC',recall_score(Y_part,Y_pred))
#         print(NN,'-F1',f1_score(Y_part,Y_pred))
        f.add_subplot(1,3,i+1)
        fpr,tpr,threshold=roc_curve(Y_part,Y_pred)
        plt.plot(fpr,tpr)
        print("NN","AUC",auc(fpr,tpr))
        print('NN','AUC_Score',roc_auc_score(Y_part,Y_pred))
    plt.show()
    return
    models=[]
    #models.append(('KNN',KNeighborsClassifier(n_neighbors=3)))
    #models.append(('GaussianNB',GaussianNB()))
    #models.append(('BernoulliNB',BernoulliNB()))
    
    models.append(('DecisionTreeGini',DecisionTreeClassifier()))
    #models.append(('DecisionTreeGini',DecisionTreeClassifier(min_impurity_split=0.1)))#效果不好舍弃
    models.append(('DecisionTreeEntropy',DecisionTreeClassifier(criterion='entropy')))
    models.append(('OriginalRandomForest',RandomForestClassifier()))
    models.append(('RandomForest',RandomForestClassifier(n_estimators=81,max_features=None))) #随机森林 max_features=None,bootstrap=False
    models.append(('AdaBoost',AdaBoostClassifier(n_estimators=1000)))#base_estimators=SVC()   algorithm='SAMME'
    models.append(('LogisticRegression',LogisticRegression(C=1000,tol=le-10,solver='saga')))#经运行，效果不如前面方法，通过调参方式
    #C越大 ，正则化越小；tol:精度；solver：方法---显然这样是不行的，罗吉斯特映射线性模型，而我们的模型不是线性可分的，所以达到这种效果
    for clf_name,clf in models:
        clf.fit(X_train,Y_train)
        xy_lst=[(X_train,Y_train),(X_validation,Y_validation),(X_test,Y_test)]#依次用0，1，2表示
        for i in range(len(xy_lst)):
            X_part=xy_lst[i][0]
            Y_part=xy_lst[i][1]
            Y_pred=clf.predict(X_part)
            print(i)
            print(clf_name,'-ACC',accuracy_score(Y_part,Y_pred))
            print(clf_name,'-REC',recall_score(Y_part,Y_pred))
            print(clf_name,'-F1',f1_score(Y_part,Y_pred))
            #dot_data=export_graphviz(clf,out_file=None,  #此时直接输出dot_data,outfile不为None,通过句柄输出
             #                       feature_names=f_names,
              #                      class_names=['NL','L'],  #标注名称
               #                     filled=True,
                #                    rounded=True,
                 #                   special_characters=True)
            #绘制树
            #graph=pydotplus.graph_from_dot_data(dot_data)
            #graph.write_pdf('dt_tree_2.pdf')
 #线性回归 
def regr_test(features,label):
    #print('X',features)
    #print('Y',label)
    from sklearn.linear_model import LinearRegression,Ridge,Lasso#分别引入线性回归，岭回归和Lasso
    #regr=LinearRegression()
    regr=Ridge(alpha=1)#alpha=0,0.2,0.4,0.8,1  效果不明显
    #regr=Lasso(alpha=0)#alpha=0,0.02  #不断调参直至参数趋于稳定
    regr.fit(features.values,label.values)
    Y_pred=regr.predict(features.values)
    print('Coef:',regr.coef_)
    from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
    print('MSE:',mean_squared_error(Y_pred,label.values))#平均平方误差
    print('MAE:',mean_absolute_error(label.values,Y_pred))
    print('R2:',r2_score(label.values,Y_pred))
def main():
    features,label=hr_preprocessing()
    #print(hr_preprocessing())
    #print(hr_processing(sl=True,le=True,npr=True,dp=True,lower_d=True,ld_n=3))
    #hr_modeling(features,label)
    regr_test(features[['number_project','average_monthly_hours']],features['last_evaluation'])
if __name__=='__main__':
    main()

Coef: [0.27247571 0.268934  ]
MSE: 0.059538014493206
MAE: 0.20435511892083338
R2: 0.16759924962620698


In [ ]:
#打印出来依次是特征、标注、参数、评估